In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [38]:
url = "https://www.worldsnowboardguide.com/resorts/search/"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(url, headers=headers)
search_page = response.text
soup = BeautifulSoup(search_page,"html5lib")

In [47]:
countries = soup.find_all(id='countryid')[0]
countries = soup.find_all('option')
list_countries = []
for c in countries:
    if "----" in c.text:
        country = c.text.replace('---- ','')
        country = country.replace(' ','').lower()
        list_countries.append(country)

In [48]:
# remove Afganistan since it was not being scraped succesfully 
list_countries.remove('afganistan')

In [41]:
def resort_meta(country_name):
    url = f"https://www.worldsnowboardguide.com/resorts/{country_name}/"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)
    search_page = response.text
    soup = BeautifulSoup(search_page,"html5lib")
    
    resorts_box = soup.find_all(id='simplesearch')[0]
    resort_names = resorts_box.find_all(class_='description-text')
    resort_links = resorts_box.find_all('a')
    
    resorts = []
    for i in resort_names:
        idx = resort_names.index(i)
        dic = {}
        dic['resort_name'] = i.find_all('h3')[0].text
        dic['resort_country'] = i.find_all('p')[0].text
        dic['resort_url'] = "https://www.worldsnowboardguide.com"+resort_links[idx].get('href')
        resorts.append(dic)

    return(resorts)

In [46]:
all_resorts = []
for c in list_countries:
    temp_list = resort_meta(c)
    all_resorts.extend(temp_list)
    print(c)

morroco
southafrica
iceland
china
india
japan
kazakhstan
northkorea
pakistan
southkorea
australia
newzealand
bulgaria
czechrepublic
estonia
georgia
hungary
kosovo
kyrgyzstan
latvia
macedonia
poland
romania
russia
serbia
slovakia
slovenia
ukraine
iran
iraq
israel
lebanon
turkey
unitedarabemirates
canada
usa
finland
norway
sweden
argentina
bolivia
chile
andorra
austria
belgium
cyprus
france
germany
greece
italy
liechtenstein
netherlands
portugal
spain
switzerland
unitedkingdom


In [51]:
df = pd.DataFrame.from_dict(all_resorts)

In [52]:
df.to_csv('worldsnowboardguide_meta.csv')